In [1]:
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.ndimage

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
CLOUD_PATH = 'C:\\Users\\norad\\ETTEPLAN\\DATA\\DATA_3F_INSIDE\\tray-b-4-d_L2_part_3_downsample_10_without_ears.ply'
base_filename = os.path.splitext(os.path.basename(CLOUD_PATH))[0]
OUTPUT_FOLDER = 'C:\\Users\\norad\\ETTEPLAN\\DATA\\DATA_PIPE_SEGMENTATION\\'
output_file_name_pipes = os.path.basename(CLOUD_PATH).replace(".ply", "_seg_pipes.ply")
output_file_name_basis = os.path.basename(CLOUD_PATH).replace(".ply", "_seg_basis.ply")
OUTPUT_PATH_PIPES = os.path.join(OUTPUT_FOLDER, output_file_name_pipes)
OUTPUT_PATH_BASIS = os.path.join(OUTPUT_FOLDER, output_file_name_basis)

In [3]:
pcd = o3d.io.read_point_cloud(CLOUD_PATH)
colors = np.asarray(pcd.colors)

In [4]:
# Convert RGB values to a scalar field (grayscale conversion based on human perception)
scalar_field = 0.2989 * colors[:, 0] + 0.5870 * colors[:, 1] + 0.1140 * colors[:, 2]

# Define threshold to separate 'main chip' from 'pipes' using the scalar field
threshold = 22.43421053 / 255  

# Identify points belonging to the 'main chip' (below the threshold) and 'pipes' (above or equal to threshold)
main_chip_points = np.where(scalar_field < threshold)
pipes_points = np.where(scalar_field >= threshold)

# Select the 'main chip' and 'pipes' point clouds based on the indices
main_chip = pcd.select_by_index(main_chip_points[0])
pipes = pcd.select_by_index(pipes_points[0])

In [5]:
# Perform DBSCAN clustering on the 'pipes' point cloud to identify connected components
labels = np.array(pipes.cluster_dbscan(eps=0.01, min_points=10))

# Get unique cluster labels and the number of points in each cluster
unique_labels, counts = np.unique(labels, return_counts=True)

# Find the label of the largest connected component (excluding noise, labeled as -1)
largest_component_label = unique_labels[counts.argmax()]

# Separate points that belong to the largest component and the smaller components
largest_component_points = np.where(labels == largest_component_label)[0]
remaining_pipes_points = np.where(labels != largest_component_label)[0]

# Select points for the largest component of the 'pipes'
remaining_points_in_pipes = pipes.select_by_index(largest_component_points)

# Select the smaller components that will be merged into 'main chip'
small_components = pipes.select_by_index(remaining_pipes_points)

# Add smaller components back to 'main chip'
main_chip += small_components


In [6]:
o3d.io.write_point_cloud(OUTPUT_PATH_BASIS, main_chip)
o3d.io.write_point_cloud(OUTPUT_PATH_PIPES, remaining_points_in_pipes)

True